# Test de la perf sur la data MNIST (chiffre) utilisant un cnn basique avec keras

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

def load_and_format_for_cnn(csv_path):

    df = pd.read_csv(csv_path)
    
    y = df['label'].values
    
    X_flat = df.iloc[:, 1:].values  # Forme actuelle : (N, 784)
    
    # 3. Normalisation (0-255 -> 0-1)
    X_flat = X_flat.astype('float32') / 255.0
    
    # LE RESHAPE MAGIQUE POUR CNN

    # 1  : Canal (Gris) Couleur, ce serait 3.
    X_cnn = X_flat.reshape(-1, 28, 28, 1)
    
    print(f"Transformation terminée : {X_flat.shape} -> {X_cnn.shape}")
    
    return X_cnn, y

X_train, y_train = load_and_format_for_cnn(Path("data/mnist_train.csv"))
X_test, y_test = load_and_format_for_cnn(Path("data/mnist_test.csv"))

Transformation terminée : (60000, 784) -> (60000, 28, 28, 1)
Transformation terminée : (10000, 784) -> (10000, 28, 28, 1)


In [ ]:
NB_CLASSES = 10

model_cnn = keras.Sequential([
    #  Détection simple (Bords, Lignes)
    keras.Input(shape=(28, 28, 1)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"), # padding="same" : Ajoute des zéros autour pour conserver la taille de l'image (28x28)
    layers.MaxPooling2D(pool_size=(2, 2)), # L'image passe de 28x28 a 14x14

    #  Détection complexe (Boucles, Formes)
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D(pool_size=(2, 2)), # L'image passe de 14x14 à 7x7

    #  Classification 
    layers.Flatten(),          # On aplatit le cube 7x7x64 en un long vecteur
    layers.Dropout(0.5),     # Pour éviter l'overfitting
    layers.Dense(NB_CLASSES, activation="softmax")
])

model_cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        31,370 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,186 (196.04 KB)

 Trainable params: 50,186 (196.04 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model_cnn.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'],
    
)

In [10]:
from tensorflow.keras import callbacks
early_stopping = callbacks.EarlyStopping(
    min_delta=0.0001, 
    patience=4, 
    restore_best_weights=True,
)

In [ ]:
history = model_cnn.fit(
    X_train, y_train,
    batch_size=64,        
    epochs=15,             
    validation_data=(X_test, y_test), #On a malheureusement pas de jeu de validation distinct par rapport au test ici
    callbacks=[early_stopping],
)

Epoch 1/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9273 - loss: 0.2358 - val_accuracy: 0.9814 - val_loss: 0.0609
Epoch 2/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9746 - loss: 0.0813 - val_accuracy: 0.9838 - val_loss: 0.0487
Epoch 3/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9804 - loss: 0.0631 - val_accuracy: 0.9865 - val_loss: 0.0390
Epoch 4/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.9831 - loss: 0.0547 - val_accuracy: 0.9881 - val_loss: 0.0329
Epoch 5/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9851 - loss: 0.0470 - val_accuracy: 0.9889 - val_loss: 0.0314
Epoch 6/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9869 - loss: 0.0421 - val_accuracy: 0.9895 - val_loss: 0.0310
Epoch 7/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9874 - loss: 0.0397 - val_accuracy: 0.9912 - val_loss: 0.0268
Epoch 8/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9881 - loss: 0.0363 - 

In [13]:
print("\n--- Évaluation sur le jeu de Test ---")
test_results = model_cnn.evaluate(X_test, y_test, batch_size=32)

print(f"Test Loss:     {test_results[0]:.4f}")
print(f"Test Accuracy: {test_results[1]*100:.2f}%")


--- Évaluation sur le jeu de Test ---
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9930 - loss: 0.0217
Test Loss:     0.0217
Test Accuracy: 99.30%
